In [2]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [13]:
# Load chunks.txt and store it in a list, separated by newlines
with open('chunks.txt', 'r') as f:
    chunks = f.read().split('\n')


In [14]:
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone

In [15]:
import os
import dotenv

dotenv.load_dotenv('env')
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
PINECONE_API_KEY = os.environ['PINECONE_API_KEY']
PINECONE_API_ENV = os.environ['PINECONE_API_ENV']

In [16]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [17]:
# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = "langchain" # put in the name of your pinecone index here

In [22]:
# docsearch = Pinecone.from_texts(chunks, embeddings, index_name=index_name) Initial population !!
docsearch = Pinecone.from_existing_index(index_name=index_name, embedding=embeddings)

In [23]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

In [24]:
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
chain = load_qa_chain(llm, chain_type="stuff")

In [25]:
query = """" Recomend a game like this:
I want to play a game with my friends, I like RPG fantasy games. It also should not be too expensive.
The magic system in the game should be good.
"""
docs = docsearch.similarity_search(query, include_metadata=True)

In [26]:
chain.run(input_documents=docs, question=query)

' OCTOPATH TRAVELER™ is a great game for you and your friends to play. It is a turn-based JRPG with 8 different stories, one for each character. The combat system is great with turns, breaks, and weaknesses. The world is also amazing and encourages exploration. The soundtrack is also incredibly atmospheric and the pixel art style is great. The price is a bit much, but it is worth it if you can get it on sale.'